In [2]:
import numpy as np
import skimage as sk
from skimage.io import imread, imsave
from glob import glob
import napari
from cellpose import io, models, train 
import os
from tqdm import tqdm

In [23]:
X = sorted(glob('../Microglia_CellPose_Model/NonEmpty_Slices/*.tif'))
X = list(map(imread,X))
Y = sorted(glob('../Microglia_CellPose_Model/NonEmpty_Masks/*.tif'))
Y = list(map(imread,Y))
rng = np.random.RandomState(42)
ind = rng.permutation(len(X))
n_val = max(1, int(round(0.15 * len(ind))))
ind_train, ind_val = ind[:-n_val], ind[-n_val:]
X_val, Y_val = [X[i] for i in ind_val]  , [Y[i] for i in ind_val]
X_trn, Y_trn = [X[i] for i in ind_train], [Y[i] for i in ind_train]

In [ ]:
X_val = np.squeeze(X_val)
Y_val = np.squeeze(Y_val)
X_trn = np.expand_dims(np.squeeze(X_trn),-1)
Y_trn = np.squeeze(Y_trn)

In [21]:
Y_trn[0].shape

(2048, 2048)

In [22]:
model = models.CellposeModel(model_type="cyto3",gpu=True)

model_path, train_losses, test_losses = train.train_seg(model.net,
                            train_data=X_trn, train_labels=Y_trn,
                            test_data=X_val, test_labels=Y_val,
                            weight_decay=0.1, learning_rate=1e-5,
                            n_epochs=100, model_name="Test")

100%|██████████| 22/22 [00:00<00:00, 37.51it/s]
52 train images with number of masks less than min_train_masks (5), removing from train set


RuntimeError: running_mean should contain 2048 elements not 2

In [24]:
save_train = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Train/'
save_test = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Test/'


In [27]:
for i in tqdm(range(len(Y_trn))):
    squeezed_Y = np.squeeze(Y_trn[i])
    squeezed_X = np.squeeze(X_trn[i])
    imsave(os.path.join(save_train,'0'+str(i)+'_masks.tif'),squeezed_Y,check_contrast=False)
    imsave(os.path.join(save_train,'0'+str(i)+'_img.tif'),squeezed_X,check_contrast=False)

for i in tqdm(range(len(Y_val))):
    squeezed_Y = np.squeeze(Y_val[i])
    squeezed_X = np.squeeze(X_val[i])
    imsave(os.path.join(save_test,'0'+str(i)+'_masks.tif'),squeezed_Y,check_contrast=False)
    imsave(os.path.join(save_test,'0'+str(i)+'_img.tif'),squeezed_X,check_contrast=False)

100%|██████████| 22/22 [00:00<00:00, 25.61it/s]


In [ ]:
viewer = napari.view_image(X_trn[0], name='img')
viewer.add_labels(Y_trn[0],name='mask')

In [28]:
train_dir = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Train/'
test_dir = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Test/'
io.logger_setup()

output = io.load_train_test_data(train_dir, test_dir, image_filter="_img",
                                mask_filter="_masks", look_one_level_down=False)
images, labels, image_names, test_images, test_labels, image_names_test = output

# e.g. retrain a Cellpose model
model = models.CellposeModel(model_type="cyto3")

model_path, train_losses, test_losses = train.train_seg(model.net,
                            train_data=images, train_labels=labels,
                            channels=[0,0], normalize=True,
                            test_data=test_images, test_labels=test_labels,
                            weight_decay=1e-4, SGD=True, learning_rate=0.1,
                            n_epochs=100, model_name="my_new_model")

creating new log file
2025-06-25 16:51:18,276 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-06-25 16:51:18,276 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-06-25 16:51:18,305 [INFO] not all flows are present, running flow generation for all images
2025-06-25 16:51:20,761 [INFO] 128 / 128 images in E:/ParadaKusz_Lab/Microglia_CellPose_Model/Train/ folder have labels
2025-06-25 16:51:20,778 [INFO] not all flows are present, running flow generation for all images
2025-06-25 16:51:21,155 [INFO] 22 / 22 images in E:/ParadaKusz_Lab/Microglia_CellPose_Model/Test/ folder have labels
2025-06-25 16:51:21,158 [INFO] >> cyto3 << model set to be used
2025-06-25 16:51:21,161 [INFO] >>>> using CPU
2025-06-25 16:51:21,162 [INFO] >>>> using CPU
2025-06-25 16:51:21,249 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\cyto3
2025-06-25 16:51:21,469 [INFO] >>>> model diam_mean =  30.00

100%|██████████| 128/128 [00:43<00:00,  2.97it/s]

2025-06-25 16:52:06,355 [INFO] computing flows for labels



100%|██████████| 22/22 [00:06<00:00,  3.33it/s]

2025-06-25 16:52:13,278 [INFO] >>> computing diameters



100%|██████████| 22/22 [00:00<00:00, 39.79it/s]

2025-06-25 16:52:16,958 [WARNING] 52 train images with number of masks less than min_train_masks (5), removing from train set
2025-06-25 16:52:17,187 [INFO] >>> using channels [0, 0]


2025-06-25 16:52:17,189 [INFO] >>> normalizing {'lowhigh': None, 'percentile': None, 'normalize': True, 'norm3D': True, 'sharpen_radius': 0, 'smooth_radius': 0, 'tile_norm_blocksize': 0, 'tile_norm_smooth3D': 1, 'invert': False}
2025-06-25 16:52:34,213 [INFO] >>> n_epochs=100, n_train=76, n_test=22
2025-06-25 16:52:34,214 [INFO] >>> SGD, learning_rate=0.10000, weight_decay=0.00010, momentum=0.900
2025-06-25 16:52:34,220 [INFO] >>> saving model to e:\ParadaKusz_Lab\OIC-136_Microglia\models\my_new_model
2025-06-25 16:53:06,692 [INFO] 0, train_loss=1.2215, test_loss=0.7580, LR=0.000000, time 32.47s
2025-06-25 16:55:34,000 [INFO] 5, train_loss=0.2134, test_loss=0.0456, LR=0.055556, time 179.78s
2025-06-25 16:57:59,550 [INFO] 10, train_loss=0.0353, test_loss=0.0393, LR=0.100000, time 325.33s
2025-06-25 17:02:49,075 [INFO] 20, train_loss=0.0335, test_loss=0.0349, LR=0.100000, time 614.86s
2025-06-25 17:07:37,008 [INFO] 30, train_loss=0.0344, test_loss=0.0331, LR=0.100000, time 902.79s
2025-0

In [29]:
img = imread("E:/ParadaKusz_Lab/TIF_Files/Control_MO/Control_MO_04.tif")

In [33]:
io.logger_setup()
model = models.CellposeModel(gpu=True,pretrained_model="E:/ParadaKusz_Lab/OIC-136_Microglia/models/my_new_model") # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'


creating new log file
2025-06-25 17:58:35,493 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-06-25 17:58:35,494 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-06-25 17:58:35,497 [INFO] ** TORCH CUDA version installed and working. **
2025-06-25 17:58:35,497 [INFO] >>>> using GPU (CUDA)
2025-06-25 17:58:35,627 [INFO] >>>> loading model E:/ParadaKusz_Lab/OIC-136_Microglia/models/my_new_model
2025-06-25 17:58:35,978 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-06-25 17:58:35,978 [INFO] >>>> model diam_labels =  40.352 (mean diameter of training ROIs)


In [43]:
channels =[[0,0]]
masks, _, _ = model.eval(img, diameter=40, channels=channels, z_axis=0,stitch_threshold=0.2, flow_threshold=1.4,cellprob_threshold=-1.0)

2025-06-25 18:10:36,722 [INFO] multi-stack tiff read in as having 33 planes 1 channels
2025-06-25 18:10:59,647 [INFO] 100%|##########| 33/33 [00:21<00:00,  1.55it/s]
2025-06-25 18:11:01,337 [INFO] network run in 23.02s
2025-06-25 18:11:19,633 [INFO] 100%|##########| 33/33 [00:18<00:00,  1.80it/s]
2025-06-25 18:11:19,635 [INFO] stitching 33 planes using stitch_threshold=0.200 to make 3D masks


100%|██████████| 32/32 [00:00<00:00, 33.87it/s]


2025-06-25 18:11:22,288 [INFO] masks created in 20.95s


In [44]:
viewer = napari.view_image(img, name='img', scale =[4.5,0.3,0.3])
viewer.add_labels(masks, name='masks',scale=[4.5,0.3,0.3])

2025-06-25 18:11:39,285 [WARNING] DirectWrite: CreateFontFaceFromHDC() failed (Indicates an error in an input file such as a font file.) for QFontDef(Family="", pointsize=12, pixelsize=16, styleHint=5, weight=50, stretch=100, hintingPreference=0) LOGFONT("MS Sans Serif", lfWidth=0, lfHeight=-16) dpi=144


<Labels layer 'masks' at 0x2259cab9990>